In [1]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen
import requests
import json
import urllib

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

from bs4 import BeautifulSoup
import re

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
credentials = {'CREDENTIALS_HERE'}
gc = gspread.authorize(credentials)

### Hubspot Data

In [2]:
REAL_apikey = 'HUBSPOT_API_KEY'

In [58]:
# import requests
# import json
# import urllib

# # hapikey = 'demo' 
# limit = 200
# comapny_list = []
# get_all_deals_url = "https://api.hubapi.com/companies/v2/companies/paged?"
# parameter_dict = {'hapikey': REAL_apikey, 'limit': limit}
# headers = {}

# # Paginate your request using offset
# has_more = True
# while has_more:
# 	parameters = urllib.parse.urlencode(parameter_dict)
# 	get_url = get_all_deals_url + parameters
# 	r = requests.get(url= get_url, headers = headers)
# 	response_dict = json.loads(r.text)
# 	has_more = response_dict['has-more']
# 	comapny_list.extend(response_dict['companies'])
# 	parameter_dict['offset']= response_dict['offset']
 
# print('loop finished')

# list_length = len(comapny_list) 

# print("You've succesfully parsed through {} company records and added them to a list".format(list_length))

# company_ids = (pd.DataFrame(comapny_list)['companyId'])

In [3]:
limit = 200
comapny_list = []
get_all_deals_url = f'https://api.hubapi.com/companies/v2/companies/paged?hapikey={REAL_apikey}&properties=ipeds_id&properties=name&properties=icp_'
parameter_dict = {'hapikey': REAL_apikey, 'limit': limit}
headers = {}

# Paginate your request using offset
has_more = True
while has_more:
	parameters = urllib.parse.urlencode(parameter_dict)
	get_url = get_all_deals_url + parameters
	r = requests.get(url= get_url, headers = headers)
	response_dict = json.loads(r.text)
	has_more = response_dict['has-more']
	comapny_list.extend(response_dict['companies'])
	parameter_dict['offset']= response_dict['offset']
 
print('loop finished')

list_length = len(comapny_list) 

print("You've succesfully parsed through {} company records and added them to a list".format(list_length))

company_ids = (pd.DataFrame(comapny_list)['companyId'])

loop finished
You've succesfully parsed through 13211 company records and added them to a list


In [55]:
# comapny_list

In [4]:
company_prop_list = []

for i in range(len(comapny_list)):
    properties_dict = {}
    comp_id = comapny_list[i]['companyId']
    properties_dict['Company ID'] = comp_id

    for key in comapny_list[i]['properties']:
        properties_dict[key] = comapny_list[i]['properties'][key]['value']

    company_prop_list.append(properties_dict)

company_df = pd.DataFrame(company_prop_list)

In [5]:
company_df

,Company ID,name,ipeds_id
0,4104270903,Ancora Education,NaN
1,4104270904,Integrishield,NaN
2,4104270905,California Healing Arts College - Carson,442985
3,4104270906,University of the Potomac - Corporate Office,NaN
4,4104270908,CyberVista,NaN
...,...,...,...
13206,8217647973,Capital Region Boces,418038
13207,8226452484,scsb.us,NaN
13208,8226502771,Emergencytraining,NaN
13209,8228328257,Group Two,NaN


In [32]:
# response.text

In [23]:
# all_companies = []

# url = f'https://api.hubapi.com/companies/v2/companies/paged?hapikey={REAL_apikey}&properties=ipeds_id&properties=name&properties=icp_'
# querystring = {"hapikey":REAL_apikey}
# headers = {
#     'Content-Type': "application/json"
#     }
# response = requests.get(url=url, params = querystring)
# response_dict = json.loads(response.text)['companies']

# properties = list(response_dict.keys())
# values = {}
# values['Company ID'] = str(company_ids[i])
# for key in properties:
#     values[key] = response_dict[key]['value']

# all_companies.append(values)

# if int(i) % 100 == 0:
#     print(i)

# all_companies_df = pd.DataFrame(all_companies)

In [19]:
# all_companies = []

# for i in range(len(company_ids)):
#     url = "https://api.hubapi.com/companies/v2/companies/"+str(company_ids[i])
#     querystring = {"hapikey":REAL_apikey}
#     headers = {
#         'Content-Type': "application/json"
#         }
#     response = requests.get(url=url, params = querystring)
#     response_dict = json.loads(response.text)['properties']

#     properties = list(response_dict.keys())
#     values = {}
#     values['Company ID'] = str(company_ids[i])
#     for key in properties:
#         values[key] = response_dict[key]['value']
    
#     all_companies.append(values)

#     if int(i) % 100 == 0:
#         print(i)
    
# all_companies_df = pd.DataFrame(all_companies)

In [10]:
# all_companies_df = pd.DataFrame(all_companies)

In [6]:
# all_companies_df.head()

In [6]:
all_companies_clean_df = company_df.fillna('')
all_companies_clean_df = all_companies_clean_df.replace(np.nan, '')
hs_with_id_df = all_companies_clean_df[all_companies_clean_df['ipeds_id']!=''].reset_index().drop('index', axis=1)

In [7]:
hs_with_id_df

,Company ID,name,ipeds_id
0,4104270905,California Healing Arts College - Carson,442985
1,4104270909,Palm Beach State College - Worth,136358
2,4104270910,Research College of Nursing - Kansas City,178989
3,4104270913,Mech - Tech College,414461
4,4104270915,Tidewater Tech - Trades,232919
...,...,...,...
6358,7529211399,Clarksburg Beauty Academy & School of Massage ...,237321
6359,7529651896,Arkansas Beauty School - Little Rock,106315
6360,7529688834,Charleston School of Beauty Culture,237303
6361,8181066351,Institute of Taoist Education and Acupuncture,454838


### Pulling from IPEDS

In [8]:
ipeds_ids = list(hs_with_id_df['ipeds_id'])
print(len(ipeds_ids))

6363


In [9]:
# ipeds_ids = ipeds_ids[0:1000]

In [9]:
### BOTH TOGETHER ###

all_accred_df = pd.DataFrame()
all_inst_accred_df = pd.DataFrame()

all_programs_df = pd.DataFrame()
program_segment = ''

link_not_work = []
no_accred = []
no_program = []


for n in range(len(ipeds_ids)):
    ipeds_id = ipeds_ids[n]
    try:
        url = 'https://nces.ed.gov/collegenavigator/?id=' + ipeds_id
    except:
        link_not_work.append(str(ipeds_id))
    
    try:
        table_MN = pd.read_html(url, match='Accreditor')

        for i in range(len(table_MN)):
            program_accred_df = pd.DataFrame()
            inst_accred_df = pd.DataFrame()

            if 'Accreditor / Program' in str(table_MN[i].columns):
                program_accred_df = table_MN[i]
                program_accred_df['Accreditor'] = ''
                program = ''
                for l in range(len(program_accred_df.columns)):
                    if 'Accreditor / Program' in str(program_accred_df.columns[l]):
                        program_accred_df = program_accred_df.rename({str(program_accred_df.columns[l]):'Program'}, axis=1)
                for j in range(len(program_accred_df)):
                    if program_accred_df['Program'][j] == program_accred_df['Status'][j]:
                        program = program_accred_df['Program'][j]
                    
                    program_accred_df['Accreditor'][j] = program
                program_accred_df = program_accred_df[program_accred_df['Status'] != program_accred_df['Accreditor']].reset_index().drop('index', axis=1)
                program_accred_df['Accred Type'] = 'Program Accreditation'
            
            else:
                inst_accred_df = table_MN[i]
                inst_accred_df['Accred Type'] = 'INSTITUTIONAL ACCREDITATION'.title()


            accred_df = pd.concat([inst_accred_df, program_accred_df]).reset_index().drop('index', axis=1)
            accred_df['ipeds_id'] = ipeds_id
            inst_accred_df['ipeds_id'] = ipeds_id

            all_accred_df = pd.concat([all_accred_df, accred_df]).reset_index().drop('index', axis=1)
            all_inst_accred_df = pd.concat([all_inst_accred_df, inst_accred_df]).reset_index().drop('index', axis=1)
    except:
        no_accred.append(str(ipeds_id))

    try:
        table_MN = pd.read_html(url, match='Program')

        for i in range(len(table_MN)):
            if 'Program' in table_MN[i].columns:
                programs_df = table_MN[i]
                programs_df['Program Segment'] = ''
                for j in range(len(programs_df)):
                    if programs_df[programs_df.columns[0]][j] == programs_df[programs_df.columns[1]][j]:
                        program_segment = programs_df[programs_df.columns[0]][j]
                    programs_df['Program Segment'][j] = program_segment
                programs_df = programs_df[programs_df['Program'] != programs_df['Program Segment']].reset_index().drop('index', axis=1)
                programs_df['ipeds_id'] = ipeds_id

                programs_df = programs_df[programs_df['Program'] == 'Category total'].reset_index().drop('index', axis=1)
                programs_df = programs_df.replace('-', '0')
                columns = programs_df.columns.drop('Program').drop('Program Segment').drop('ipeds_id')

                programs_df['Total'] = 0
                for k in range(len(programs_df)):
                    sum_total = 0
                    for c in columns:
                        sum_total += int(programs_df[c][k].replace('d', ''))
                    programs_df['Total'][k] = sum_total
                programs_df = programs_df.drop(columns, axis = 1).drop('Program', axis=1)
                largest_programs_df = programs_df[programs_df['Total']==programs_df['Total'].max()]
        
        all_programs_df = pd.concat([all_programs_df, largest_programs_df]).reset_index().drop('index', axis=1)
    except:
        no_program.append(str(ipeds_id))
    
    if n % 100 == 0:
        print(n)
    

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [49]:
only_program_accred = all_accred_df[all_accred_df['Accred Type'] == 'Program Accreditation'].reset_index().drop('index', axis=1)
only_program_accred_reindex = only_program_accred.reindex(['Accreditor', 'ipeds_id'], axis=1).drop_duplicates().reset_index().drop('index', axis=1)
only_program_accred_list = only_program_accred_reindex.groupby('ipeds_id')['Accreditor'].apply(list).reset_index()

In [50]:
only_program_accred_list

,ipeds_id,Accreditor
0,100654,"[Academy of Nutrition and Dietetics, Accredita..."
1,100663,"[American Dental Association, Commission on De..."
2,100690,[Commission on Accrediting of the Association ...
3,100706,"[Commission on Collegiate Nursing Education, N..."
4,100724,"[American Occupational Therapy Association, Ac..."
...,...,...
2584,492962,"[American Bar Association, Council of the Sect..."
2585,493488,[Accreditation Commission for Education in Nur...
2586,494436,[Joint Review Committee on Education in Radiol...
2587,494843,"[American Dental Association, Commission on De..."


In [24]:
for_hs_accred_df = all_inst_accred_df.groupby('ipeds_id')['Accreditor'].apply(list).reset_index()
final_accreditor_df = hs_with_id_df.merge(for_hs_accred_df, on='ipeds_id', how = 'outer')
final_df = final_accreditor_df.merge(all_programs_df, on='ipeds_id', how = 'outer').drop('Total', axis=1)

In [25]:
final_df

,Company ID,name,ipeds_id,Accreditor,Program Segment
0,4104270905,California Healing Arts College - Carson,442985,[Accrediting Commission of Career Schools and ...,Agricultural/Animal/Plant/Veterinary Science a...
1,4104270909,Palm Beach State College - Worth,136358,"[Southern Association of Colleges and Schools,...","Liberal Arts and Sciences, General Studies and..."
2,4104270910,Research College of Nursing - Kansas City,178989,[Higher Learning Commission],Health Professions and Related Programs
3,4104270913,Mech - Tech College,414461,[Accrediting Council for Continuing Education ...,Mechanic and Repair Technologies/Technicians
4,4104270915,Tidewater Tech - Trades,232919,[Council on Occupational Education],Precision Production
...,...,...,...,...,...
6455,7529211399,Clarksburg Beauty Academy & School of Massage ...,237321,[National Accrediting Commission of Career Art...,"Culinary, Entertainment, and Personal Services"
6456,7529651896,Arkansas Beauty School - Little Rock,106315,[National Accrediting Commission of Career Art...,"Culinary, Entertainment, and Personal Services"
6457,7529688834,Charleston School of Beauty Culture,237303,[National Accrediting Commission of Career Art...,"Culinary, Entertainment, and Personal Services"
6458,8181066351,Institute of Taoist Education and Acupuncture,454838,[Accreditation Commission for Acupuncture and ...,Health Professions and Related Programs


In [20]:
all_programs_df

,Program Segment,ipeds_id,Total
0,Agricultural/Animal/Plant/Veterinary Science a...,442985,64
1,"Liberal Arts and Sciences, General Studies and...",136358,4062
2,Health Professions and Related Programs,178989,203
3,Mechanic and Repair Technologies/Technicians,414461,802
4,Precision Production,232919,224
...,...,...,...
5797,"Culinary, Entertainment, and Personal Services",237321,21
5798,"Culinary, Entertainment, and Personal Services",106315,73
5799,"Culinary, Entertainment, and Personal Services",237303,33
5800,Health Professions and Related Programs,454838,4


In [17]:
final_df.to_csv('IPEDS_info.csv')

In [21]:
final_df

,Company ID,name,ipeds_id,Accreditor,Program Segment
0,4104270905,California Healing Arts College - Carson,442985,Accrediting Commission of Career Schools and C...,Agricultural/Animal/Plant/Veterinary Science a...
1,4104270909,Palm Beach State College - Worth,136358,"Southern Association of Colleges and Schools, ...","Liberal Arts and Sciences, General Studies and..."
2,4104270910,Research College of Nursing - Kansas City,178989,Higher Learning Commission,Health Professions and Related Programs
3,4104270913,Mech - Tech College,414461,Accrediting Council for Continuing Education a...,Mechanic and Repair Technologies/Technicians
4,4104270915,Tidewater Tech - Trades,232919,Council on Occupational Education,Precision Production
...,...,...,...,...,...
5639,7527983016,Chris' Beauty College,175494,National Accrediting Commission of Career Arts...,"Culinary, Entertainment, and Personal Services"
5640,7529211399,Clarksburg Beauty Academy & School of Massage ...,237321,National Accrediting Commission of Career Arts...,"Culinary, Entertainment, and Personal Services"
5641,7529651896,Arkansas Beauty School - Little Rock,106315,National Accrediting Commission of Career Arts...,"Culinary, Entertainment, and Personal Services"
5642,7529688834,Charleston School of Beauty Culture,237303,National Accrediting Commission of Career Arts...,"Culinary, Entertainment, and Personal Services"


In [38]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1wQrZ0VbuVxgN4rwG6gP8UVsWk3XfPa6EpndSm9508Yk/edit#gid=1874121679')
sheet = wb.worksheet('Sheet8')

In [27]:
final_clean_df = final_df

In [28]:
for i in range(len(final_clean_df)):
    final_clean_df['Accreditor'][i] = str(final_clean_df['Accreditor'][i]).replace('[', '').replace(']', '').replace("'", '')
    


In [52]:
set_with_dataframe(sheet, final_clean_df)

In [18]:
final_clean_df.to_csv('IPEDS_clean_info.csv')

In [16]:
set(list(final_df['Program Segment']))

{'Agricultural/Animal/Plant/Veterinary Science and Related Fields',
 'Architecture and Related Services',
 'Area, Ethnic, Cultural, Gender, and Group Studies',
 'Biological and Biomedical Sciences',
 'Business, Management, Marketing, and Related Support Services',
 'Communication, Journalism, and Related Programs',
 'Communications Technologies/Technicians and Support Services',
 'Computer and Information Sciences and Support Services',
 'Construction Trades',
 'Culinary, Entertainment, and Personal Services',
 'Education',
 'Engineering',
 'Engineering/Engineering-related Technologies/Technicians',
 'English Language and Literature/Letters',
 'Family and Consumer Sciences/Human Sciences',
 'Foreign Languages, Literatures, and Linguistics',
 'Health Professions and Related Programs',
 'History',
 'Homeland Security, Law Enforcement, Firefighting and Related Protective Services',
 'Legal Professions and Studies',
 'Liberal Arts and Sciences, General Studies and Humanities',
 'Mathematic

Student count

In [77]:
ipeds_ids_test = [442985, 136358, 178989, 414461, 232919]

In [83]:
all_general_df = pd.DataFrame()
no_general_info = []

for i in range(len(ipeds_ids)):
    ipeds_id = ipeds_ids[i]
    url = 'https://nces.ed.gov/collegenavigator/?id=' + str(ipeds_id)
    try:
        table_MN = pd.read_html(url, match='General')

        general_info_df = table_MN[0].transpose()
        general_info_df.columns = general_info_df.iloc[0]
        general_info_df = general_info_df[1:]
        general_info_df['ipeds_id'] = ipeds_id

        all_general_df = pd.concat([all_general_df, general_info_df]).reset_index().drop('index', axis=1)
    except:
        no_general_info.append(ipeds_id)
        
    if i % 100 == 0:
        print(i)

In [179]:
ipeds_ids_test

[442985, 136358, 178989, 414461, 232919]

In [190]:
no_name = []
name_df = pd.DataFrame()
all_names_df = pd.DataFrame()

for i in range(len(ipeds_ids)):
    ipeds_id = ipeds_ids[i]
    url = 'https://nces.ed.gov/collegenavigator/?id=' + str(ipeds_id)
    try:
        r = requests.get(url= url)
        ipeds_raw = BeautifulSoup(r.text)
        school_name = ipeds_raw.find('title').text
        name_df = pd.DataFrame({'School Name':[school_name], 'ipeds_id':[ipeds_id]})
        all_names_df = pd.concat([all_names_df, name_df])
    except:
        no_name.append(ipeds_id)
        
    if i % 100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300


In [191]:
all_names_df

,School Name,ipeds_id
0,\r\n\tCollege Navigator - California Healing A...,442985
0,\r\n\tCollege Navigator - Palm Beach State Col...,136358
0,\r\n\tCollege Navigator - Research College of ...,178989
0,\r\n\tCollege Navigator - Mech-Tech College\r\n,414461
0,\r\n\tCollege Navigator - Tidewater Tech-Trade...,232919
...,...,...
0,\r\n\tCollege Navigator - Clarksburg Beauty Ac...,237321
0,\r\n\tCollege Navigator - Arkansas Beauty Scho...,106315
0,\r\n\tCollege Navigator - Charleston School of...,237303
0,\r\n\tCollege Navigator - Institute of Taoist ...,454838


In [193]:
all_names_clean_df = all_names_df.reset_index().drop('index', axis=1)

In [197]:
all_names_clean_df['School Name'][0]

'\r\n\tCollege Navigator - California Healing Arts College\r\n'

In [201]:
for i in range(len(all_names_clean_df)):
    all_names_clean_df['School Name'][i] = str(all_names_clean_df['School Name'][i]).replace('\r', '').replace('\n', '').replace('\t', '').replace('College Navigator - ', '')
    

In [202]:
all_names_clean_df

,School Name,ipeds_id
0,California Healing Arts College,442985
1,Palm Beach State College,136358
2,Research College of Nursing,178989
3,Mech-Tech College,414461
4,Tidewater Tech-Trades,232919
...,...,...
6357,Clarksburg Beauty Academy and School of Massag...,237321
6358,Arkansas Beauty School-Little Rock,106315
6359,Charleston School of Beauty Culture,237303
6360,Institute of Taoist Education and Acupuncture,454838


In [203]:
names_sheet = wb.worksheet('Sheet11')
set_with_dataframe(names_sheet, all_names_clean_df)

In [158]:
url = 'https://nces.ed.gov/collegenavigator/?id=' + str(ipeds_id)

In [173]:
r = requests.get(url= url)
ipeds_raw = BeautifulSoup(r.text)

In [178]:
ipeds_raw.find('title').text

'College Navigator - Albany BOCES-Adult Practical Nursing Program'

In [172]:
print(BeautifulSoup(r.text).prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head id="ctl00_hd">
  <meta content="text/html;charset=utf-8" http-equiv="Content-type"/>
  <title>
   College Navigator - Albany BOCES-Adult Practical Nursing Program
  </title>
  <link href="css/md0.css" rel="stylesheet" type="text/css"/>
  <meta content="noindex,nofollow" name="robots"/>
  <link href="css/collegenav.css?v=2.4" rel="stylesheet" type="text/css"/>
  <link href="css/mapstyle.css" rel="stylesheet" type="text/css"/>
  <!--[if lte IE 8]><link href="css/mainIE6.css" text="text/css" rel="stylesheet" /><![endif]-->
  <link href="http://nces.ed.gov/collegenavigator/opensearch.ashx?md=0" rel="search" title="NCES College Navigator" type="application/opensearchdescription+xml"/>
 </head>
 <body id="ctl00_bodyMain" onload="if(typeof imgPL=='function')imgPL();if(typeof sp=='function')sp();">
 

In [84]:
all_general_df

,General information:,Website:,Type:,Awards offered:,Campus setting:,Campus housing:,Student population:,Student-to-faculty ratio:,ipeds_id,Related Institutions:
0,(310) 826-7622,www.chac.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Small,No,181 (all undergraduate),13 to 1,442985,NaN
1,(561) 967-7222,www.palmbeachstate.edu/,"4-year, primarily associate's, Public",Less than one year certificateOne but less tha...,Suburb: Large,No,"26,666 (all undergraduate)",29 to 1,136358,NaN
2,(816) 995-2800,researchcollege.edu/,"4-year, Private for-profit",Bachelor's degreeMaster's degreePost-master's ...,City: Large,Yes,420 (269 undergraduate),8 to 1,178989,NaN
3,(787) 744-1060 x1000,www.mechtech.edu/,"2-year, Private for-profit",One but less than two years certificateAssocia...,City: Small,No,"1,947 (all undergraduate)",30 to 1,414461,NaN
4,(757) 858-8324,www.tidewatertechtrades.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Midsize,No,436 (all undergraduate),12 to 1,232919,NaN
...,...,...,...,...,...,...,...,...,...,...
5863,(304) 624-6475 x110,clarksburgbeautyacademywv.com/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,Town: Distant,No,47 (all undergraduate),12 to 1,237321,NaN
5864,(501) 562-5673,arbs.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Midsize,No,15 (all undergraduate),20 to 1,106315,NaN
5865,(304) 346-9603,charlestonschoolofbeautyculture.com/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Small,No,62 (all undergraduate),17 to 1,237303,NaN
5866,(720) 890-8922,www.itea.edu/,"4-year, Private not-for-profit",Master's degree,Suburb: Small,No,28 (0 undergraduate),NaN,454838,NaN


In [85]:
import re

In [87]:
all_general_clean_df = all_general_df.rename({'General information:':'Phone Number', 'Website:':'Website', 'Type:':'School Type', 'Awards offered:':'Degree Type', 'Campus setting:':'Campus Setting', 'Campus housing:':'Campus Housing', 'Student population:':'Total Students', 'Student-to-faculty ratio:':'Student-to-faculty ratio'}, axis=1)

for i in range(len(all_general_clean_df)):
    all_general_clean_df['Total Students'][i] = re.sub("\(.*?\)", '', str(all_general_clean_df['Total Students'][i]))

In [88]:
all_general_clean_df

,Phone Number,Website,School Type,Degree Type,Campus Setting,Campus Housing,Total Students,Student-to-faculty ratio,ipeds_id,Related Institutions:
0,(310) 826-7622,www.chac.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Small,No,181,13 to 1,442985,NaN
1,(561) 967-7222,www.palmbeachstate.edu/,"4-year, primarily associate's, Public",Less than one year certificateOne but less tha...,Suburb: Large,No,"26,666",29 to 1,136358,NaN
2,(816) 995-2800,researchcollege.edu/,"4-year, Private for-profit",Bachelor's degreeMaster's degreePost-master's ...,City: Large,Yes,420,8 to 1,178989,NaN
3,(787) 744-1060 x1000,www.mechtech.edu/,"2-year, Private for-profit",One but less than two years certificateAssocia...,City: Small,No,"1,947",30 to 1,414461,NaN
4,(757) 858-8324,www.tidewatertechtrades.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Midsize,No,436,12 to 1,232919,NaN
...,...,...,...,...,...,...,...,...,...,...
5863,(304) 624-6475 x110,clarksburgbeautyacademywv.com/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,Town: Distant,No,47,12 to 1,237321,NaN
5864,(501) 562-5673,arbs.edu/,"< 2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Midsize,No,15,20 to 1,106315,NaN
5865,(304) 346-9603,charlestonschoolofbeautyculture.com/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Small,No,62,17 to 1,237303,NaN
5866,(720) 890-8922,www.itea.edu/,"4-year, Private not-for-profit",Master's degree,Suburb: Small,No,28,NaN,454838,NaN


In [89]:
general_sheet = wb.worksheet('General Info')
set_with_dataframe(general_sheet, all_general_clean_df)

In [107]:
all_general_clean_df = all_general_clean_df.fillna('')
parent_colleges = all_general_clean_df[all_general_clean_df['Related Institutions:']!=""].reset_index().drop('index', axis=1)

In [125]:
parent_colleges

,Phone Number,Website,School Type,Degree Type,Campus Setting,Campus Housing,Total Students,Student-to-faculty ratio,ipeds_id,Related Institutions:
0,(817) 226-1900 x228,www.ccitraining.edu/,"< 2-year, Private for-profit",Less than one year certificate,City: Large,No,200,12 to 1,449773,CCI Training Center
1,(502) 968-4858,www.americancollegeofbarbering.com/,"< 2-year, Private for-profit",One but less than two years certificate,City: Large,No,78,20 to 1,486169,American College of Barbering - Florence
2,(561) 381-4990,www.cambridgehealth.edu/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Small,No,654,25 to 1,454865,Cambridge College of Healthcare & Technology
3,(620) 251-7700,www.coffeyville.edu/,"2-year, Public",Less than one year certificateOne but less tha...,Town: Remote,Yes,"1,368",21 to 1,154925,"Coffeyville Technical Campus, Columbus Technic..."
4,(718) 793-2330,nysmda.com/,"2-year, Private for-profit",Less than one year certificateOne but less tha...,City: Large,No,381,20 to 1,193858,NYSMedical&Dental Assistants - American Instit...
...,...,...,...,...,...,...,...,...,...,...
187,(310) 954-4000,www.msmu.edu/,"4-year, Private not-for-profit",Less than one year certificateAssociate's degr...,City: Large,Yes,"2,745",10 to 1,119173,Mount Saint Mary's University - Doheny Campus
188,(787) 787-9661,www.lestoncollege.com/,"< 2-year, Private for-profit",One but less than two years certificate,City: Midsize,No,18,20 to 1,444732,Leston College -
189,(800) 861-6400,www.myunion.edu/,"4-year, Private not-for-profit",Less than one year certificateBachelor's degre...,City: Large,No,809,7 to 1,206279,"Florida Center, Los Angeles Center, Sacramento..."
190,(512) 404-4800,www.austinseminary.edu/,"4-year, Private not-for-profit",Master's degreeDoctor's degree - professional ...,City: Large,Yes,181,,223001,Austin Presbyterian Theological Seminary- YMCA...


In [126]:

for j in range(len(parent_colleges)):
    parent_colleges['Related Institutions:'][j] = str(parent_colleges['Related Institutions:'][j]).replace('This is the parent institution of:', '')
    text = parent_colleges['Related Institutions:'][j]
    new_text = ''
    for i, letter in enumerate(text):
        if i and letter.isupper() and text[i-1].islower():
            new_text += ', '

        new_text += letter
    
    parent_colleges['Related Institutions:'][j] = new_text

In [136]:
parent_colleges['Related Institutions:'][3].split(', ')

['Coffeyville Technical Campus', 'Columbus Technical Campus']

In [140]:
parent_dict = {}
for i in range(len(parent_colleges)):
    child_list = parent_colleges['Related Institutions:'][i].split(', ')
    for j in range(len(child_list)):
        parent_dict[child_list[j]] = parent_colleges['ipeds_id'][i]


In [155]:
parent_df = pd.DataFrame(list(parent_dict.items()),columns = ['School Name','Parent ipeds_id']) 


In [156]:
parent_df

,School Name,Parent ipeds_id
0,CCI Training Center,449773
1,American College of Barbering - Florence,486169
2,Cambridge College of Healthcare & Technology,454865
3,Coffeyville Technical Campus,154925
4,Columbus Technical Campus,154925
...,...,...
489,Florida Center,206279
490,Los Angeles Center,206279
491,Sacramento Center,206279
492,Austin Presbyterian Theological Seminary- YMCA...,223001


In [157]:
parent_sheet = wb.worksheet('Parents')
set_with_dataframe(parent_sheet, parent_df)

In [58]:
### ONLY ACCRED ###

all_accred_df = pd.DataFrame()
all_inst_accred_df = pd.DataFrame()

for n in range(len(ipeds_ids)):
    ipeds_id = ipeds_ids[n]
    try:
        url = 'https://nces.ed.gov/collegenavigator/?id=' + ipeds_id
        table_MN = pd.read_html(url, match='Accreditor')

        for i in range(len(table_MN)):
            program_accred_df = pd.DataFrame()
            inst_accred_df = pd.DataFrame()

            if 'Accreditor / Program' in str(table_MN[i].columns):
                program_accred_df = table_MN[i]
                program_accred_df['Accreditor'] = ''
                program = ''
                for l in range(len(program_accred_df.columns)):
                    if 'Accreditor / Program' in str(program_accred_df.columns[l]):
                        program_accred_df = program_accred_df.rename({str(program_accred_df.columns[l]):'Program'}, axis=1)
                for j in range(len(program_accred_df)):
                    if program_accred_df['Program'][j] == program_accred_df['Status'][j]:
                        program = program_accred_df['Program'][j]
                    
                    program_accred_df['Accreditor'][j] = program
                program_accred_df = program_accred_df[program_accred_df['Status'] != program_accred_df['Accreditor']].reset_index().drop('index', axis=1)
                program_accred_df['Accred Type'] = 'Program Accreditation'
            
            else:
                inst_accred_df = table_MN[i]
                inst_accred_df['Accred Type'] = 'INSTITUTIONAL ACCREDITATION'.title()


            accred_df = pd.concat([inst_accred_df, program_accred_df]).reset_index().drop('index', axis=1)
            accred_df['ipeds_id'] = ipeds_id
            inst_accred_df['ipeds_id'] = ipeds_id

            all_accred_df = pd.concat([all_accred_df, accred_df]).reset_index().drop('index', axis=1)
            all_inst_accred_df = pd.concat([all_inst_accred_df, inst_accred_df]).reset_index().drop('index', axis=1)
    except:
        print(ipeds_id)

440226
483717


In [ ]:
all_accred_df

,Accreditor,Status,Next Review Date,Accred Type,ipeds_id,Program
0,Accrediting Commission of Career Schools and C...,Accredited since (01/27/2002),02/28/2022,Institutional Accreditation,442985,NaN
1,"Southern Association of Colleges and Schools, ...",Accredited since (01/01/1942),06/14/2022,Institutional Accreditation,136358,NaN
2,"Academy of Nutrition and Dietetics, Accreditat...",Accredited (06/01/1983 - 12/31/2006),NaN,Program Accreditation,136358,Dietetics (DIETT) - Dietetic Technician Program
3,Accreditation Commission for Education in Nurs...,Accredited since (06/01/1996),06/30/2025,Program Accreditation,136358,Nursing (ADNUR) - Associate degree programs
4,Accreditation Commission for Education in Nurs...,Accredited since ((!)10/01/2014),12/31/2027,Program Accreditation,136358,Nursing (NUR) - Baccalaureate program
...,...,...,...,...,...,...
298,Middle States Commission on Higher Education,Accredited since ((!)01/01/1983),01/01/2026,Institutional Accreditation,214582,NaN
299,"American Physical Therapy Association, Commiss...",Accredited since (04/29/2015),06/30/2030,Program Accreditation,214582,Physical Therapy (PTA) - Programs for the phys...
300,"American Physical Therapy Association, Commiss...",Preaccredited (04/30/2014 - 04/29/2015),NaN,Program Accreditation,214582,Physical Therapy (PTA) - Programs for the phys...
301,WASC Senior College and University Commission,Accredited since (02/28/1977),06/30/2023,Institutional Accreditation,112394,NaN


In [ ]:
all_inst_accred_df

,Accreditor,Status,Next Review Date,Accred Type,ipeds_id
0,Accrediting Commission of Career Schools and C...,Accredited since (01/27/2002),02/28/2022,Institutional Accreditation,442985
1,"Southern Association of Colleges and Schools, ...",Accredited since (01/01/1942),06/14/2022,Institutional Accreditation,136358
2,Higher Learning Commission,Accredited since ((!)01/01/1987),08/31/2022,Institutional Accreditation,178989
3,Accrediting Council for Continuing Education a...,Accredited since ((!)08/01/1990),08/30/2024,Institutional Accreditation,414461
4,Council on Occupational Education,Accredited since (11/11/1988),11/11/2022,Institutional Accreditation,232919
...,...,...,...,...,...
108,Council on Occupational Education,Accredited since (02/20/2012),12/31/2020,Institutional Accreditation,481368
109,Council on Occupational Education,Preaccredited (06/04/2011 - 02/20/2012),NaN,Institutional Accreditation,481368
110,Middle States Commission on Higher Education,Accredited since ((!)01/01/1983),01/01/2026,Institutional Accreditation,214582
111,WASC Senior College and University Commission,Accredited since (02/28/1977),06/30/2023,Institutional Accreditation,112394


In [ ]:
for_hs_accred_df = all_inst_accred_df.groupby('ipeds_id')['Accreditor'].apply(list).reset_index()

In [ ]:
final_accreditor_df = hs_with_id_df.merge(for_hs_accred_df, on='ipeds_id')

In [ ]:
final_accreditor_df

,Company ID,name,ipeds_id,Accreditor
0,4104270905,California Healing Arts College - Carson,442985,[Accrediting Commission of Career Schools and ...
1,4104270909,Palm Beach State College - Worth,136358,"[Southern Association of Colleges and Schools,..."
2,4104270910,Research College of Nursing - Kansas City,178989,[Higher Learning Commission]
3,4104270913,Mech - Tech College,414461,[Accrediting Council for Continuing Education ...
4,4104270915,Tidewater Tech - Trades,232919,[Council on Occupational Education]
...,...,...,...,...
89,4104275518,Midland College - Midland,226806,"[Southern Association of Colleges and Schools,..."
90,4104275521,Grossmont College - Cajon,115296,"[Western Association of Schools and Colleges, ..."
91,4104275522,Prestige Health & Beauty Sciences Academy - Mi...,481368,"[Council on Occupational Education, Council on..."
92,4104275525,Pennsylvania Institute of Technology - Media,214582,[Middle States Commission on Higher Education]


In [ ]:
### ONLY PROGRAMS

all_programs_df = pd.DataFrame()
program_segment = ''

for n in range(len(ipeds_ids)):
    ipeds_id = ipeds_ids[n]
    try:
        url = 'https://nces.ed.gov/collegenavigator/?id=' + ipeds_id
        table_MN = pd.read_html(url, match='Program')

        for i in range(len(table_MN)):
            if 'Program' in table_MN[i].columns:
                programs_df = table_MN[i]
                programs_df['Program Segment'] = ''
                for j in range(len(programs_df)):
                    if programs_df[programs_df.columns[0]][j] == programs_df[programs_df.columns[1]][j]:
                        program_segment = programs_df[programs_df.columns[0]][j]
                    programs_df['Program Segment'][j] = program_segment
                programs_df = programs_df[programs_df['Program'] != programs_df['Program Segment']].reset_index().drop('index', axis=1)
                programs_df['ipeds_id'] = ipeds_id

                programs_df = programs_df[programs_df['Program'] == 'Category total'].reset_index().drop('index', axis=1)
                programs_df = programs_df.replace('-', '0')
                columns = programs_df.columns.drop('Program').drop('Program Segment').drop('ipeds_id')

                programs_df['Total'] = 0
                for k in range(len(programs_df)):
                    sum_total = 0
                    for c in columns:
                        sum_total += int(programs_df[c][k].replace('d', ''))
                    programs_df['Total'][k] = sum_total
                programs_df = programs_df.drop(columns, axis = 1).drop('Program', axis=1)
                largest_programs_df = programs_df[programs_df['Total']==programs_df['Total'].max()]
        
        all_programs_df = pd.concat([all_programs_df, largest_programs_df]).reset_index().drop('index', axis=1)
    except:
        print(ipeds_id)

483717
454218
457703


In [49]:
all_programs_df

,Program Segment,ipeds_id,Total
0,Agricultural/Animal/Plant/Veterinary Science a...,442985,64
1,"Liberal Arts and Sciences, General Studies and...",136358,4062
2,Health Professions and Related Programs,178989,203
3,Mechanic and Repair Technologies/Technicians,414461,802
4,Precision Production,232919,224
...,...,...,...
92,"Liberal Arts and Sciences, General Studies and...",226806,222
93,"Liberal Arts and Sciences, General Studies and...",115296,1607
94,"Culinary, Entertainment, and Personal Services",481368,57
95,Health Professions and Related Programs,214582,92


In [52]:
final_df = final_accreditor_df.merge(all_programs_df, on='ipeds_id').drop('Total', axis=1)

In [53]:
final_df

,Company ID,name,ipeds_id,Accreditor,Program Segment
0,4104270905,California Healing Arts College - Carson,442985,[Accrediting Commission of Career Schools and ...,Agricultural/Animal/Plant/Veterinary Science a...
1,4104270909,Palm Beach State College - Worth,136358,"[Southern Association of Colleges and Schools,...","Liberal Arts and Sciences, General Studies and..."
2,4104270910,Research College of Nursing - Kansas City,178989,[Higher Learning Commission],Health Professions and Related Programs
3,4104270913,Mech - Tech College,414461,[Accrediting Council for Continuing Education ...,Mechanic and Repair Technologies/Technicians
4,4104270915,Tidewater Tech - Trades,232919,[Council on Occupational Education],Precision Production
...,...,...,...,...,...
89,4104275518,Midland College - Midland,226806,"[Southern Association of Colleges and Schools,...","Liberal Arts and Sciences, General Studies and..."
90,4104275521,Grossmont College - Cajon,115296,"[Western Association of Schools and Colleges, ...","Liberal Arts and Sciences, General Studies and..."
91,4104275522,Prestige Health & Beauty Sciences Academy - Mi...,481368,"[Council on Occupational Education, Council on...","Culinary, Entertainment, and Personal Services"
92,4104275525,Pennsylvania Institute of Technology - Media,214582,[Middle States Commission on Higher Education],Health Professions and Related Programs


In [56]:
set(list(final_df['Program Segment']))

{'Agricultural/Animal/Plant/Veterinary Science and Related Fields',
 'Business, Management, Marketing, and Related Support Services',
 'Communications Technologies/Technicians and Support Services',
 'Computer and Information Sciences and Support Services',
 'Culinary, Entertainment, and Personal Services',
 'Engineering/Engineering-related Technologies/Technicians',
 'Health Professions and Related Programs',
 'Homeland Security, Law Enforcement, Firefighting and Related Protective Services',
 'Liberal Arts and Sciences, General Studies and Humanities',
 'Mechanic and Repair Technologies/Technicians',
 'Precision Production',
 'Visual and Performing Arts'}